# Libraries and Data

In [40]:
# Libraries
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from greykite.framework.templates.autogen.forecast_config import *
from greykite.framework.templates.forecaster import Forecaster
from greykite.framework.templates.model_templates import ModelTemplateEnum
from greykite.common.features.timeseries_features import *
from greykite.common.evaluation import EvaluationMetricEnum
from greykite.framework.utils.result_summary import summarize_grid_search_results
from plotly.offline import iplot

In [47]:
# Import Data
df = pd.read_csv('../nyc_data.csv')
future_df = pd.read_csv('../future.csv')
future_df.head()

,Date,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
0,1/1/2021,NaN,0,0,0,5.00,154.221
1,1/2/2021,NaN,0,0,0,11.11,264.805
2,1/3/2021,NaN,0,0,0,3.89,115.499
3,1/4/2021,NaN,0,0,0,6.67,124.650
4,1/5/2021,NaN,0,0,0,5.56,77.968


In [6]:
# Merge both
df = pd.concat([df, future_df])
df = df.reset_index(drop=True)
df.tail(1)

,Date,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
2222,1/31/2021,NaN,0,0,0,4.44,158.62


In [7]:
# Rename
df = df.rename(columns={"Demand": 'y'})
df.head(0)

,Date,y,Easter,Thanksgiving,Christmas,Temperature,Marketing


# Silverkit Preps

In [8]:
# Specifying Time Series names
metadata = MetadataParam(time_col="Date",
                        value_col="y",
                        freq="D",
                        train_end_date=pd.to_datetime("2020-12-31"))
metadata

MetadataParam(anomaly_info=None, date_format=None, freq='D', time_col='Date', train_end_date=Timestamp('2020-12-31 00:00:00'), value_col='y')

In [9]:
# Growth Term Possibilities
growth = dict(growth_term = ["linear", 'quadratic', "sqrt"])
growth

{'growth_term': ['linear', 'quadratic', 'sqrt']}

In [10]:
# Seasonalities
seaonality = dict(yearly_seasonality = "auto",
                  quarterly_seasonality = "auto",
                  monthly_seasonality= "auto",
                  weekly_seasonality = "auto",
                  daily_seasonality = "auto")

seaonality

{'yearly_seasonality': 'auto',
 'quarterly_seasonality': 'auto',
 'monthly_seasonality': 'auto',
 'weekly_seasonality': 'auto',
 'daily_seasonality': 'auto'}

In [11]:
# Checking which countries are available and holidays
get_available_holiday_lookup_countries(["US"])
get_available_holidays_across_countries(countries=['US'],
                                        year_start=2015,
                                        year_end=2021)


['Christmas Day',
 'Christmas Day (Observed)',
 'Columbus Day',
 'Halloween',
 'Independence Day',
 'Independence Day (Observed)',
 'Juneteenth National Independence Day',
 'Juneteenth National Independence Day (Observed)',
 'Labor Day',
 'Martin Luther King Jr. Day',
 'Memorial Day',
 "New Year's Day",
 "New Year's Day (Observed)",
 'Thanksgiving',
 'Veterans Day',
 'Veterans Day (Observed)',
 "Washington's Birthday"]

In [12]:
# Specifying Events
events = dict(holidays_to_model_separately = ["New Year's Day"],
              holiday_lookup_countries = ["US"],
              holiday_pre_num_days = 2,
              holiday_post_num_days = 2,
              holiday_pre_post_num_dict = {"New Year's Day": (3,1)},
              daily_event_df_dict = {"elections": pd.DataFrame({
                  "date": ["2016-11-08", "2020-11-03"],
                  "event_name": ["elections"] * 2
              })})
events

{'holidays_to_model_separately': ["New Year's Day"],
 'holiday_lookup_countries': ['US'],
 'holiday_pre_num_days': 2,
 'holiday_post_num_days': 2,
 'holiday_pre_post_num_dict': {"New Year's Day": (3, 1)},
 'daily_event_df_dict': {'elections':          date event_name
  0  2016-11-08  elections
  1  2020-11-03  elections}}

In [13]:
# Changepoints -> reflects the changes in trend
changepoints = dict(changepoints_dict = dict(method = "auto"))
changepoints

{'changepoints_dict': {'method': 'auto'}}

In [14]:
# Regressors
regressors = dict(regressor_cols = ["Easter", "Temperature", "Marketing"])
regressors

{'regressor_cols': ['Easter', 'Temperature', 'Marketing']}

In [15]:
 # Lagged Regressors
lagged_regressors = dict(lagged_regressor_dict = {"Temperature": "auto",
                                                   "Easter": "auto",
                                                   "Marketing": "auto"})
lagged_regressors

{'lagged_regressor_dict': {'Temperature': 'auto',
  'Easter': 'auto',
  'Marketing': 'auto'}}

In [16]:
# Autoregression - > dependant on forecasting horizon
autoregression = dict(autoreg_dict = "auto")
autoregression

{'autoreg_dict': 'auto'}

In [17]:
# Fitting Algorithms
custom = dict(fit_algorithm_dict = [dict(fit_algorithm = "linear"),
                               dict(fit_algorithm = "ridge"),
                               dict(fit_algorithm = "rf"),
                               dict(fit_algorithm = "gradient_boosting")])
custom

{'fit_algorithm_dict': [{'fit_algorithm': 'linear'},
  {'fit_algorithm': 'ridge'},
  {'fit_algorithm': 'rf'},
  {'fit_algorithm': 'gradient_boosting'}]}

# Silverkite Model

In [18]:
# Build the model
model_components = ModelComponentsParam(growth=growth,
                                        seasonality=seaonality,
                                        events=events,
                                        changepoints=changepoints,
                                        regressors=regressors,
                                        lagged_regressors=lagged_regressors,
                                        autoregression=autoregression,
                                        custom=custom)



In [19]:
df.shape[0] - 180 - 31

2012

In [20]:
# Cross-validation
evaluation_period = EvaluationPeriodParam(cv_min_train_periods=df.shape[0] - 180 - 31,
                                          cv_expanding_window=True,
                                          cv_max_splits=50,
                                          cv_periods_between_splits=16)

In [21]:
# Evaluation Metric
evaluation_metric = EvaluationMetricParam(cv_selection_metric=EvaluationMetricEnum.RootMeanSquaredError.name)

In [22]:
# Configuration
config = ForecastConfig(model_template=ModelTemplateEnum.SILVERKITE.name,
                        forecast_horizon=31,
                        metadata_param=metadata,
                        model_components_param=model_components,
                        evaluation_period_param=evaluation_period,
                        evaluation_metric_param=evaluation_metric)

In [23]:
# Forecasting
forecaster = Forecaster()
result = forecaster.run_forecast_config(df = df,
                                        config=config)

Fitting 8 folds for each of 12 candidates, totalling 96 fits


C:\Users\foxhe\AppData\Roaming\Python\Python310\site-packages\greykite\algo\forecast\silverkite\forecast_silverkite.py:3433: UserWarning:

The following Fourier series terms are removed due to collinearity:
['sin1_toq_quarterly', 'cos1_toq_quarterly', 'sin2_toq_quarterly', 'cos2_toq_quarterly', 'sin3_toq_quarterly', 'cos3_toq_quarterly', 'cos4_tow_weekly']

C:\Users\foxhe\AppData\Roaming\Python\Python310\site-packages\greykite\algo\forecast\silverkite\forecast_silverkite.py:3433: UserWarning:

The following Fourier series terms are removed due to collinearity:
['sin1_toq_quarterly', 'cos1_toq_quarterly', 'sin2_toq_quarterly', 'cos2_toq_quarterly', 'sin3_toq_quarterly', 'cos3_toq_quarterly', 'cos4_tow_weekly']

C:\Users\foxhe\AppData\Roaming\Python\Python310\site-packages\greykite\algo\forecast\silverkite\forecast_silverkite.py:3433: UserWarning:

The following Fourier series terms are removed due to collinearity:
['sin1_toq_quarterly', 'cos1_toq_quarterly', 'sin2_toq_quarterly', 'cos2_

In [24]:
# Visualization
fig = result.backtest.plot()
iplot(fig)

# Parameter Tuning Results

In [ ]:
# CV results
cv_results = summarize_grid_search_results(grid_search=result.grid_search,
                                           decimals=1,
                                           score_func=EvaluationMetricEnum.RootMeanSquaredError.name)


In [29]:
# Set CV results index
cv_results["params"] = cv_results["params"].astype(str)
cv_results.set_index("params", drop=True, inplace=True)
cv_results

,rank_test_CORR,rank_test_R2,rank_test_MSE,rank_test_RMSE,rank_test_MAE,rank_test_MedAE,rank_test_MAPE,rank_test_MedAPE,rank_test_sMAPE,rank_test_Q80,...,std_test_OutsideTolerance5p,split0_train_OutsideTolerance5p,split1_train_OutsideTolerance5p,split2_train_OutsideTolerance5p,split3_train_OutsideTolerance5p,split4_train_OutsideTolerance5p,split5_train_OutsideTolerance5p,split6_train_OutsideTolerance5p,split7_train_OutsideTolerance5p,std_train_OutsideTolerance5p
params,,,,,,,,,,,,,,,,,,,,,
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",2,2,2,2,1,3,1,3,1,7,...,0.1,0.5,0.5,0.4,0.4,0.4,0.4,0.4,0.4,0.0
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",1,1,1,1,2,1,2,1,2,5,...,0.1,0.5,0.4,0.4,0.4,0.4,0.4,0.4,0.4,0.0
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",3,3,3,3,3,2,3,2,3,6,...,0.1,0.5,0.5,0.4,0.4,0.4,0.4,0.4,0.4,0.0
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",5,6,6,6,6,6,5,6,6,12,...,0.0,0.4,0.5,0.4,0.4,0.4,0.4,0.4,0.4,0.0
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",4,4,4,4,4,4,4,4,4,10,...,0.1,0.4,0.5,0.4,0.4,0.4,0.4,0.4,0.4,0.0
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",6,5,5,5,5,5,6,5,5,11,...,0.0,0.4,0.5,0.4,0.4,0.4,0.4,0.4,0.4,0.0
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'rf'})]",11,11,10,11,11,11,11,11,11,8,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.0
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'rf'})]",12,12,12,12,12,10,12,10,12,9,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.0
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'rf'})]",10,10,11,10,10,12,10,12,10,4,...,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.1,0.0


In [30]:
# Looking at the best results
cv_results[["rank_test_RMSE", "mean_test_RMSE", 
            "param_estimator__fit_algorithm_dict",
            "param_estimator__growth_term"]]

,rank_test_RMSE,mean_test_RMSE,param_estimator__fit_algorithm_dict,param_estimator__growth_term
params,,,,
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",2,47.8,{'fit_algorithm': 'linear'},linear
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",1,47.8,{'fit_algorithm': 'linear'},quadratic
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]",3,47.9,{'fit_algorithm': 'linear'},sqrt
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",6,48.9,{'fit_algorithm': 'ridge'},linear
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",4,48.5,{'fit_algorithm': 'ridge'},quadratic
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'ridge'})]",5,48.9,{'fit_algorithm': 'ridge'},sqrt
"[('estimator__growth_term', 'linear'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'rf'})]",11,64.9,{'fit_algorithm': 'rf'},linear
"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'rf'})]",12,66.1,{'fit_algorithm': 'rf'},quadratic
"[('estimator__growth_term', 'sqrt'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'rf'})]",10,64.9,{'fit_algorithm': 'rf'},sqrt


In [34]:
# Best Params
best_params = cv_results[cv_results.rank_test_RMSE == 1][["mean_test_RMSE", 
                                            "param_estimator__fit_algorithm_dict",
                                            "param_estimator__growth_term"]].transpose()
best_params

params,"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]"
mean_test_RMSE,47.8
param_estimator__fit_algorithm_dict,{'fit_algorithm': 'linear'}
param_estimator__growth_term,quadratic


In [35]:
best_params.to_csv("SilverkiteBestParams.csv")

# Forecasting

In [36]:
# Load up best parameters
parameters = pd.read_csv('SilverkiteBestParams.csv', index_col=0)
parameters

,"[('estimator__growth_term', 'quadratic'), ('estimator__fit_algorithm_dict', {'fit_algorithm': 'linear'})]"
mean_test_RMSE,47.8
param_estimator__fit_algorithm_dict,{'fit_algorithm': 'linear'}
param_estimator__growth_term,quadratic


In [39]:
# Get the parameters
growth_term_param = parameters.loc["param_estimator__growth_term"][0]
fit_algorithm_param = parameters.loc["param_estimator__fit_algorithm_dict"][0]

In [48]:
df = pd.concat([df, future_df])
df = df.reset_index(drop=True)
df.head()

,Date,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
0,1/1/2015,720.000885,0,0,0,3.68,41.305
1,1/2/2015,581.276773,0,0,0,4.73,131.574
2,1/3/2015,754.117039,0,0,0,7.23,162.700
3,1/4/2015,622.252774,0,0,0,10.96,160.281
4,1/5/2015,785.373319,0,0,0,6.92,51.077


In [49]:
# Inspect tail
df.tail()

,Date,Demand,Easter,Thanksgiving,Christmas,Temperature,Marketing
2218,1/27/2021,NaN,0,0,0,3.33,39.664
2219,1/28/2021,NaN,0,0,0,1.67,195.314
2220,1/29/2021,NaN,0,0,0,-2.78,235.894
2221,1/30/2021,NaN,0,0,0,1.11,152.752
2222,1/31/2021,NaN,0,0,0,4.44,158.620


In [50]:
# Rename
df = df.rename(columns={"Demand": 'y'})
df.head(0)

,Date,y,Easter,Thanksgiving,Christmas,Temperature,Marketing


In [51]:
# Specifying Time Series names
metadata = MetadataParam(time_col="Date",
                        value_col="y",
                        freq="D",
                        train_end_date=pd.to_datetime("2020-12-31"))
metadata

MetadataParam(anomaly_info=None, date_format=None, freq='D', time_col='Date', train_end_date=Timestamp('2020-12-31 00:00:00'), value_col='y')

In [52]:
growth = dict(growth_term = growth_term_param)
growth

{'growth_term': 'quadratic'}

In [53]:
# Seasonalities
seaonality = dict(yearly_seasonality = "auto",
                  quarterly_seasonality = "auto",
                  monthly_seasonality= "auto",
                  weekly_seasonality = "auto",
                  daily_seasonality = "auto")

seaonality

{'yearly_seasonality': 'auto',
 'quarterly_seasonality': 'auto',
 'monthly_seasonality': 'auto',
 'weekly_seasonality': 'auto',
 'daily_seasonality': 'auto'}

In [54]:
# Specifying Events
events = dict(holidays_to_model_separately = ["New Year's Day"],
              holiday_lookup_countries = ["US"],
              holiday_pre_num_days = 2,
              holiday_post_num_days = 2,
              holiday_pre_post_num_dict = {"New Year's Day": (3,1)},
              daily_event_df_dict = {"elections": pd.DataFrame({
                  "date": ["2016-11-08", "2020-11-03"],
                  "event_name": ["elections"] * 2
              })})
events

{'holidays_to_model_separately': ["New Year's Day"],
 'holiday_lookup_countries': ['US'],
 'holiday_pre_num_days': 2,
 'holiday_post_num_days': 2,
 'holiday_pre_post_num_dict': {"New Year's Day": (3, 1)},
 'daily_event_df_dict': {'elections':          date event_name
  0  2016-11-08  elections
  1  2020-11-03  elections}}

In [55]:
# Changepoints -> reflects the changes in trend
changepoints = dict(changepoints_dict = dict(method = "auto"))
changepoints

{'changepoints_dict': {'method': 'auto'}}

In [56]:
# Regressors
regressors = dict(regressor_cols = ["Easter", "Temperature", "Marketing"])
regressors

{'regressor_cols': ['Easter', 'Temperature', 'Marketing']}

In [57]:
 # Lagged Regressors
lagged_regressors = dict(lagged_regressor_dict = {"Temperature": "auto",
                                                   "Easter": "auto",
                                                   "Marketing": "auto"})
lagged_regressors

{'lagged_regressor_dict': {'Temperature': 'auto',
  'Easter': 'auto',
  'Marketing': 'auto'}}

In [58]:
# Autoregression - > dependant on forecasting horizon
autoregression = dict(autoreg_dict = "auto")
autoregression

{'autoreg_dict': 'auto'}

In [61]:
# Fitting Algorithms
import yaml
custom = dict(fit_algorithm_dict = yaml.load(fit_algorithm_param, Loader=yaml.FullLoader))
custom

{'fit_algorithm_dict': {'fit_algorithm': 'linear'}}

# Silverkite Model

In [69]:
# Build the model
model_components = ModelComponentsParam(growth=growth,
                                        seasonality=seaonality,
                                        events=events,
                                        changepoints=changepoints,
                                        regressors=regressors,
                                        lagged_regressors=lagged_regressors,
                                        autoregression=autoregression,
                                        custom=custom)


In [70]:
# Cross-validation
evaluation_period = EvaluationPeriodParam(cv_min_train_periods=df.shape[0] - 180 - 31,
                                          cv_expanding_window=True,
                                          cv_max_splits=50,
                                          cv_periods_between_splits=16)

In [71]:
# Evaluation Metric
evaluation_metric = EvaluationMetricParam(cv_selection_metric=EvaluationMetricEnum.RootMeanSquaredError.name)

In [72]:
# Configuration
config = ForecastConfig(model_template=ModelTemplateEnum.SILVERKITE.name,
                        forecast_horizon=31,
                        metadata_param=metadata,
                        model_components_param=model_components,
                        evaluation_period_param=evaluation_period,
                        evaluation_metric_param=evaluation_metric)

In [73]:
# Forecasting
forecaster = Forecaster()
result = forecaster.run_forecast_config(df = df,
                                        config=config)

Fitting 8 folds for each of 1 candidates, totalling 8 fits


C:\Users\foxhe\AppData\Roaming\Python\Python310\site-packages\greykite\algo\forecast\silverkite\forecast_silverkite.py:3433: UserWarning:

The following Fourier series terms are removed due to collinearity:
['sin1_toq_quarterly', 'cos1_toq_quarterly', 'sin2_toq_quarterly', 'cos2_toq_quarterly', 'sin3_toq_quarterly', 'cos3_toq_quarterly', 'cos4_tow_weekly']

C:\Users\foxhe\AppData\Roaming\Python\Python310\site-packages\greykite\algo\forecast\silverkite\forecast_silverkite.py:3433: UserWarning:

The following Fourier series terms are removed due to collinearity:
['sin1_toq_quarterly', 'cos1_toq_quarterly', 'sin2_toq_quarterly', 'cos2_toq_quarterly', 'sin3_toq_quarterly', 'cos3_toq_quarterly', 'cos4_tow_weekly']

C:\Users\foxhe\AppData\Roaming\Python\Python310\site-packages\greykite\algo\forecast\silverkite\forecast_silverkite.py:3433: UserWarning:

The following Fourier series terms are removed due to collinearity:
['sin1_toq_quarterly', 'cos1_toq_quarterly', 'sin2_toq_quarterly', 'cos2_

In [67]:
# Model summary
summary = result.model[-1].summary() # -1 retrieves the estimator from pipeline
print(summary)

================================ Model Summary =================================

Number of observations: 2192,   Number of features: 182
Method: Ordinary least squares
Number of nonzero features: 182

Residuals:
         Min           1Q       Median           3Q          Max
      -164.5       -32.26      -0.7283        30.86        233.8

            Pred_col Estimate Std. Err  t value Pr(>|t|) sig. code              95%CI
           Intercept    66.76    3.005    22.21   <2e-16       ***     (60.87, 72.66)
events_New Years Day   -15.42    22.44  -0.6872    0.492              (-59.42, 28.58)
 events_N...rs Day-1   -44.55    22.57   -1.974    0.049         *  (-88.82, -0.2823)
 events_N...rs Day-2   -7.228    22.61  -0.3197    0.749              (-51.57, 37.11)
 events_N...rs Day-3   -40.49    22.55   -1.795    0.073         .    (-84.72, 3.745)
 events_N...rs Day+1   -2.608    22.37  -0.1166    0.907              (-46.47, 41.25)
        events_Other   -6.222    6.759  -0.9206    0.3

In [74]:
# Visualization
fig = result.forecast.plot_components()
iplot(fig)

# Exporting forecasts


In [79]:
# Getting result
forecast = result.forecast.df[["ts", "forecast"]]
forecast = forecast.rename(columns={'forecast': 'silverkite'})
preds_sk = forecast.iloc[-31:]
preds_sk

,ts,silverkite
2192,2021-01-01,749.870698
2193,2021-01-02,834.345095
2194,2021-01-03,751.784844
2195,2021-01-04,898.242799
2196,2021-01-05,778.078463
2197,2021-01-06,916.174178
2198,2021-01-07,821.268544
2199,2021-01-08,753.050830
2200,2021-01-09,822.383670
2201,2021-01-10,735.551425


In [80]:
# export
preds_sk.to_csv('predictions_silverkite.csv')